# Simple discrete-event simulator "from scratch" #

To see in more detail how a discrete-event simulation engine works, let's build one from scratch in Python.

In any discrete-event simulation, we have the following core components:

* **Time** refers to that in the **simulation** (simulated world), unless otherwise specified
* An **event** is a timestamped callback function that maps an initial state to a new state
* The **future event list** is a global priority queue of events, ordered by (logical) time
* The **simulation engine** runs a loop that extracts each event and executes it

These constitute a generic simulation engine, which is _independent_ of the "physical" system we might be simulating. Let's start by implementing each one in turn.

## Setup or helper code ##

In [ ]:
from pprint import pprint # pretty-printing basic data structures

In [ ]:
# https://stackoverflow.com/questions/279561/what-is-the-python-equivalent-of-static-variables-inside-a-function
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

## Time ##

Let's assume there is a global clock that maintains the current time in the simulation. This clock may be accessed via the function, `now()`.

In [ ]:
@static_vars(t=0)
def now():
    return now.t

print("The current simulation time is", now(), "o'clock.")

> Note: `static_vars` is a custom decorator that creates a static variable (locally defined, globally existing) with the given initial value. It's analogous to the `static` keyword in C/C++/Java. See [this link](https://stackoverflow.com/questions/279561/what-is-the-python-equivalent-of-static-variables-inside-a-function) for details.

Let's assume two additional helper functions for moving this global clock forward, as well as resetting it.

In [ ]:
def set_time(t_new=0):
    now.t = t_new
    return now()

In [ ]:
# Demo:
set_time(5)
print("Now it's", now(), "o'clock.")

In [ ]:
set_time() # no arg => reset
print("Can we have a do-over, please? It's now", now(), "o'clock.")

## Event type ##

Let's implement an **event** as a _timestamped_ function. Let's define the `Event` type to be a simple wrapper class with two fields:

* `t`: the **timestamp** of the event; and
* `f`: the **event handler**, which is a callback function that implements the event. (_Note:_ `f` will map a _(state, event list)_ pair to a new state, but let's defer the exact interface for now.)

In [ ]:
from dataclasses import dataclass, field
from typing import Callable

@dataclass(order=True)
class Event:
    t: int
    f: Callable=field(compare=False) # ??? — just a sec!

In [ ]:
# Demo:
e = Event(3, lambda _1=None, _2=None: "hello, world")
print(e.t, e.f())

**Comparing events.** Recall the funny type annotation attached to the function field, `f`:

```python
    t: int
    f: Callable=field(compare=False) ### ???
```

Its purpose is to allow the comparison of two events by timestamp only.

In [ ]:
e1 = Event(3, lambda _1=None, _2=None: "me first")
e2 = Event(7, lambda _1=None, _2=None: "me second")

print("Is `e1 < e2`? ==>", e1 < e2)

## Future event list ##

We need a priority queue data structure for the future event list. The simplest one is, arguably, Python's [`heapq` module](https://docs.python.org/3/library/heapq.html#priority-queue-implementation-notes).

Let's wrap it in a basic iterator API.

In [ ]:
class FutureEventList:
    def __init__(self):
        self.events = []
        
    def __iter__(self):
        return self
    
    def __next__(self) -> Event:
        from heapq import heappop
        if self.events:
            return heappop(self.events)
        raise StopIteration
    
    def __repr__(self) -> str:
        from pprint import pformat
        return pformat(self.events)

In [ ]:
# Demo:
event_list = FutureEventList()
event_list

In addition, let's define a function to "schedule" (insert) a new event into the event list.

In [ ]:
def schedule(e: Event, fev: FutureEventList):
    from heapq import heappush
    heappush(fev.events, e)

In [ ]:
# Recall:
print("Recall events:")
print("* `e1`:", e1)
print("* `e2`:", e2)

In [ ]:
# Insert them, but in _reverse_ chronological order:
schedule(e2, event_list)
schedule(e1, event_list)

In [ ]:
print("\n* Did it work? Let's see:\n")
print(event_list)

In addition, the choice to implement an iterator API makes event loops easy!

In [ ]:
event_list = FutureEventList()
schedule(e2, event_list)
schedule(e1, event_list)
print(event_list)

In [ ]:
for e in event_list:
    print(e)

## Simulation engine (executive) ##

The last piece of infrastructure we need is the _simulation engine_ or _simulation executive_.

In [ ]:
def simulate(state, event_list, verbose=True):
    for e in event_list:
        set_time(e.t)
        state = e.f(state, event_list)
        if verbose: print(f"[t={e.t}] {state}")

Here, we see the precise interface for event handlers: they map an initial state _and_ an event list to a new state.

It must also accept an event list since **events can schedule new events.**

Here's a quick demo. Recall the signature of `simulate`:

```python
def simulate(state, event_list, ...):
    ...
```

In [ ]:
event_list = FutureEventList() # Example event list
schedule(e2, event_list)
schedule(e1, event_list)
print(event_list)

In [ ]:
print(f"* Initial event list:\n{event_list}")
print("\n* Let's go!\n")
simulate({}, event_list)

## Example simulation: Airport model ##

![Summary of the airport model](airport-event-summary.png)

In [ ]:
R = 3  # Time using the runway
G = 4  # Time on the ground

In [ ]:
def initial_state():
    return {'in_air': 0,         # No. of aircraft landing or waiting to do so
            'on_ground': 0,      # No. of landed aircraft (on the ground, prior to departing)
            'runway_free': True} # Is runway free?

print(initial_state())

![Summary of the airport model](airport-event-summary.png)

We need three types of events: `arrived`, `landed`, and `departed`.

* **Arrived:** An aircraft has arrived at the airport. If the runway is free, it can land by occupying the runway and scheduling a landing event `R` timesteps in the future. Otherwise, it should wait in the (logical) "in-air" queue.

![Summary of the airport model](airport-event-summary.png)

* **Landed**: An aircraft has landed. It no longer needs the runway. It can now sit on the ground and schedule a `departed` event for `G` timesteps later. It can also schedule a landing event for any other in-air aircraft.

![Summary of the airport model](airport-event-summary.png)

* **Departed:** An aircraft has "left the building" (airport). It's no longer on the ground. Also recall that we are only modeling arrival runway traffic, assuming any number of aircraft can leave without additional waiting.

![Summary of the airport model](airport-event-summary.png)

In [ ]:
def arrived(s, fev):
    s['in_air'] += 1
    if s['runway_free']:
        s['runway_free'] = False
        schedule(Event(now() + R, landed), fev)
    return s

![Summary of the airport model](airport-event-summary.png)

In [ ]:
def landed(s, fev):
    assert not s['runway_free']
    s['in_air'] -= 1
    s['on_ground'] += 1
    schedule(Event(now() + G, departed), fev)
    if s['in_air']:
        schedule(Event(now() + R, landed), fev)
    else:
        s['runway_free'] = True
    return s

![Summary of the airport model](airport-event-summary.png)

In [ ]:
def departed(s, fev):
    s['on_ground'] -= 1
    return s

## Let's simulate! ##

In [ ]:
event_list = FutureEventList()
state = initial_state()

schedule(Event(1, arrived), event_list)
schedule(Event(3, arrived), event_list)

In [ ]:
simulate(state, event_list)

In [ ]:
## (end demo) ##